In [16]:
!python3 --version

Python 3.8.10


In [ ]:
from pyspark.sql import SparkSession

In [77]:
import time
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

def subgraph2vec(spark_session, data_file_path=None):
    schema = StructType([
        StructField("source", IntegerType(), True),
        StructField("target", IntegerType(), True),
        StructField("edge_type", StringType(), True)
    ])

    data_file_path = "/opt/spark/data/large_graph.txt" # Giữ nguyên path cố định hoặc dùng tham số
    
    df = spark_session.read.option("delimiter", "|").option("header", "false") \
        .schema(schema).csv(data_file_path)
    
    total_records = df.count()
    print(f"📋 Total edges processed: {total_records:,}")

    df_repartitioned = df.repartition(10)
    def process_partition_and_write_out(partition_iterator):
        result = []
        for record in partition_iterator:
            time.sleep(0.0000001)  # Giả lập xử lý
            print('Hihi')
            result.append(record)
        return iter(result)

    df_processed = df_repartitioned.rdd.mapPartitions(process_partition_and_write_out)

    return df_processed.toDF(schema)

In [78]:

spark = None
try:
    spark = SparkSession.builder \
        .appName("SimpleGraphProcessing_v3") \
        .master("spark://spark-master:7077") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    graph_df = subgraph2vec(spark)
    graph_df.show()
    print("✅ Graph processing completed successfully!")
except Exception as e:
    import traceback
    traceback.print_exc()    
finally:
    if spark:
        spark.stop()

📋 Total edges processed: 100,001


+------+------+----------+
|source|target| edge_type|
+------+------+----------+
|  8694|  7821|  mentions|
|  1113|  8944|    shares|
|  9672|  4447|     likes|
|  3382|  7517|     likes|
|  9992|  9887|     likes|
|  4230|  1933|friendship|
|  5799|  8606|  mentions|
|  2897|  9272|     likes|
|  8993|  2904|    shares|
|  8072|  7247|     likes|
|  1351|  2990|   follows|
|  7557|  8492|     likes|
|  8855|   558|friendship|
|  2718|  6229|    shares|
|  6075|  3298|friendship|
|  4221|  1780|   follows|
|  1449|  4656|  mentions|
|  8932|  1867|     likes|
|   934|  5901|  mentions|
|  3179|  4066|  mentions|
+------+------+----------+
only showing top 20 rows

✅ Graph processing completed successfully!
